# Clustering: Clustering Conteo de Lineas por Cluster

In [ ]:
import pandas as pd
import numpy as np

import datetime as dt
import matplotlib.dates as mdates

from pyspark import SparkContext, SQLContext
from pyspark.ml.feature import VectorAssembler

from pyspark.sql import SQLContext
from pyspark.sql.types import *


sqlCtx = SQLContext(sc)

## 1. Carga de Datos

In [2]:
#Carga de los datos para los dias Laborables
df_count_lineas_laborables=sqlContext.sql("select linea,sum(n_viajeros) as n_viajeros from emt_smartbus.viajeros_tramos_master_nooutliers where festivo=0 and \
                                  (dia_semana='L' or dia_semana='M' or dia_semana='X' or dia_semana='J' or dia_semana='V') group by linea")

df_count_lineas_festivos=sqlContext.sql("select linea,sum(n_viajeros) as n_viajeros from emt_smartbus.viajeros_tramos_master_nooutliers where \
                                  (dia_semana='S' or dia_semana='D' or festivo=1) group by linea")

df_clusters_laborables=sqlContext.sql("select * from emt_smartbus.clustering1_laborables")

df_clusters_festivos=sqlContext.sql("select * from emt_smartbus.clustering1_festivos")


In [14]:
df_count_laborables=df_count_lineas_laborables.join(df_clusters_laborables, 'linea', 'inner')
df_count_festivos=df_count_lineas_festivos.join(df_clusters_festivos, 'linea', 'inner')

In [13]:
df_count_laborables.show()

+-----+----------+-------+
|linea|n_viajeros|cluster|
+-----+----------+-------+
|  148|   4495600|      5|
|  516|    260139|      2|
|   31|  10272868|      0|
|   85|   5758030|      0|
|  451|    415965|      0|
|  137|   3235104|      0|
|   65|   3394930|      0|
|  799|    292858|      3|
|  481|   1415681|      0|
|   53|   5918146|      0|
|  133|   6777454|      0|
|  513|    286269|      2|
|   78|   2967742|      0|
|  155|   3971575|      0|
|  108|   1066894|      5|
|   34|  13539214|      0|
|  101|    429832|      1|
|  115|   5067876|      0|
|  756|      9791|      1|
|  126|   3569171|      0|
+-----+----------+-------+
only showing top 20 rows



In [29]:
df_count_laborables.registerTempTable("count_laborables")
df_count_festivos.registerTempTable("count_festivos")

In [31]:
df_result_laborables = sqlContext.sql('select linea,sum(n_viajeros) as nviajeros,cluster from count_laborables group by linea,cluster order by nviajeros desc')
df_result_festivos = sqlContext.sql('select linea,sum(n_viajeros) as nviajeros,cluster from count_festivos group by linea,cluster order by nviajeros desc')

In [32]:
df_result_laborables.registerTempTable("count_result_laborables")
df_result_festivos.registerTempTable("count_result_festivos")

In [48]:
df_rest_laborables = sqlContext.sql('select linea,nviajeros,cluster from count_result_laborables where cluster=5 order by nviajeros desc')
df_rest_festivos = sqlContext.sql('select linea,nviajeros,cluster from count_result_festivos where cluster=5 order by nviajeros desc')

In [49]:
df_rest_laborables.show()

+-----+---------+-------+
|linea|nviajeros|cluster|
+-----+---------+-------+
|   32|  8274177|      5|
|   10|  8201793|      5|
|  124|  7049076|      5|
|  143|  5769879|      5|
|  118|  5411725|      5|
|   66|  5392717|      5|
|    3|  5358157|      5|
|   54|  4765118|      5|
|  102|  4760698|      5|
|   26|  4562003|      5|
|  148|  4495600|      5|
|   24|  4457072|      5|
|  103|  3834784|      5|
|   57|  3801867|      5|
|  142|  3693242|      5|
|  141|  3690372|      5|
|  136|  3294908|      5|
|  149|  3225749|      5|
|  130|  3218993|      5|
|  119|  3157189|      5|
+-----+---------+-------+
only showing top 20 rows



In [50]:
df_rest_festivos.show()

+-----+---------+-------+
|linea|nviajeros|cluster|
+-----+---------+-------+
|  110|   187359|      5|
|  702|    94932|      5|
|  172|    61361|      5|
|  799|    46493|      5|
|  704|    32176|      5|
|  728|    19804|      5|
|  742|     1891|      5|
|  732|      776|      5|
+-----+---------+-------+

